In [1]:
import sys, os, glob
import json
import argparse
import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
import torch.optim as optim
from torch.optim import lr_scheduler

import smplx
from human_body_prior.tools.model_loader import load_vposer

from utils import ContinousRotReprDecoder, GeometryTransformer, BodyParamParser

import time

import os
import pickle
import random

In [2]:
def recover_global_T(x_batch, cam_intrisic, max_depth):
    xt_batch = x_batch[:,:3]
    xr_batch = x_batch[:,3:]

    fx_batch = cam_intrisic[:,0,0]
    fy_batch = cam_intrisic[:,1,1]
    px_batch = cam_intrisic[:,0,2]
    py_batch = cam_intrisic[:,1,2]
    s_ = 1.0 / torch.max(px_batch, py_batch)

    z = (xt_batch[:, 2]+1.0)/2.0 * max_depth

    x = xt_batch[:,0] * z / s_ / fx_batch
    y = xt_batch[:,1] * z / s_ / fy_batch
    
    xt_batch_recoverd = torch.stack([x,y,z],dim=-1)

    return torch.cat([xt_batch_recoverd, xr_batch],dim=-1)



def convert_to_3D_rot(x_batch):
    xt = x_batch[:,:3]
    xr = x_batch[:,3:9]
    xb = x_batch[:,9:]

    xr_mat = ContinousRotReprDecoder.decode(xr) # return [:,3,3]
    xr_aa = ContinousRotReprDecoder.matrot2aa(xr_mat) # return [:,3]

    return torch.cat([xt, xr_aa, xb], dim=-1)


def body_params_encapsulate(x_body_rec, to_numpy=True, batched=False):
    
    if to_numpy:
        x_body_rec_np = x_body_rec.detach().cpu().numpy()
    else:
        x_body_rec_np = x_body_rec
        
    
    if batched:
        body_params_batch_rec={}
        body_params_batch_rec['transl'] = x_body_rec_np[:,:3]
        body_params_batch_rec['global_orient'] = x_body_rec_np[:,3:6]
        body_params_batch_rec['betas'] = x_body_rec_np[:,6:16]
        body_params_batch_rec['body_pose'] = x_body_rec_np[:,16:48]
        #body_params_batch_rec['left_hand_pose'] = x_body_rec_np[:,48:60]
        #body_params_batch_rec['right_hand_pose'] = x_body_rec_np[:,60:]
        
        return body_params_batch_rec
    
    else:
        n_batch = x_body_rec_np.shape[0]
        rec_list = []

        for b in range(n_batch):
            body_params_batch_rec={}
            body_params_batch_rec['transl'] = x_body_rec_np[b:b+1,:3]
            body_params_batch_rec['global_orient'] = x_body_rec_np[b:b+1,3:6]
            body_params_batch_rec['betas'] = x_body_rec_np[b:b+1,6:16]
            body_params_batch_rec['body_pose'] = x_body_rec_np[b:b+1,16:48]
            #body_params_batch_rec['left_hand_pose'] = x_body_rec_np[b:b+1,48:60]
            #body_params_batch_rec['right_hand_pose'] = x_body_rec_np[b:b+1,60:]
            rec_list.append(body_params_batch_rec)

        return rec_list




def data_preprocessing(img, modality, target_domain_size=[128, 128]):

    """
    input:
        - img (depthmap or semantic map): [height, width].
        - modality: 'depth' or 'seg'
    output:
        canvas: with shape of target_domain_size, where the input is in the
                center tightly, with shape target_domain_size
        factor: the resizing factor
    """

    # prepare the canvas
    img_shape_o = img.shape
    canvas = torch.zeros([1,1]+target_domain_size, dtype=torch.float32,
                         device=torch.device(device))


    # filter out unavailable values
    if modality == 'depth':
        img[img>6.0]=6.0

    if modality == 'seg':
        img[img>41] = 41



    ## rescale to [-1,1]
    max_val = torch.max(img)
    _img = 2* img / max_val - 1.0

    ## put _img to the canvas
    if img_shape_o[0]>= img_shape_o[1]:
        factor = float(target_domain_size[0]) / img_shape_o[0]
        target_height = target_domain_size[0]
        target_width = int(img_shape_o[1] * factor) //2 *2 

        # for depth map we use bilinear interpolation in resizing
        # for segmentation map we use bilinear interpolation as well.
        # note that float semantic label is not real in practice, but
        # helpful in our work
        target_size = [target_height, target_width]

        _img = _img.view(1,1,img_shape_o[0],img_shape_o[1])
        img_resize = F.interpolate(_img, size=target_size, mode='bilinear',
                                    align_corners=False)

        na = target_width
        nb = target_domain_size[1]
        lower = (nb //2) - (na //2)
        upper = (nb //2) + (na //2)

        canvas[:,:,:, lower:upper] = img_resize


    else:
        factor = float(target_domain_size[1]) / img_shape_o[1]

        target_height = int(factor*img_shape_o[0]) //2 *2
        target_width = target_domain_size[1]

        target_size = [target_height, target_width]
        _img = _img.view(1,1,img_shape_o[0],img_shape_o[1])
        img_resize = F.interpolate(_img, size=target_size, mode='bilinear',
                                    align_corners=False)

        na = target_height
        nb = target_domain_size[0]
        lower = (nb //2) - (na //2)
        upper = (nb //2) + (na //2)

        canvas[:,:,lower:upper, :] = img_resize

    return canvas, factor, max_val


device="cpu"

def scipy_matfile_parse(filename):
    '''
    parse data from files and put them to GPU
    Note that this function is for demo, and is different from the ones used in other places.
    '''
    data = sio.loadmat(filename)
    depth0_np = data['depth']
    seg0_np = data['seg']

    ## change them to torch tensor
    depth0 = torch.tensor(depth0_np, dtype=torch.float32, device=torch.device(device))
    seg0 = torch.tensor(seg0_np, dtype=torch.float32, device=torch.device(device))

    ## pre_processing
    depth, factor_d,max_d = data_preprocessing(depth0, 'depth', target_domain_size=[128, 128])
    seg, factor_s,_ = data_preprocessing(seg0, 'seg', target_domain_size=[128, 128])


    cam_intrinsic_np = data['cam'][0][0]['intrinsic']
    cam_intrinsic = torch.tensor(cam_intrinsic_np, dtype=torch.float32, device=torch.device(device)).unsqueeze(0)
    cam_extrinsic_np = data['cam'][0][0]['extrinsic']
    cam_extrinsic_np = np.linalg.inv(cam_extrinsic_np)
    cam_extrinsic = torch.tensor(cam_extrinsic_np, dtype=torch.float32, device=torch.device(device)).unsqueeze(0)

    return depth, seg, max_d.view(1), cam_intrinsic, cam_extrinsic

In [4]:
def create_dataset(data):
    dataset=[]
    for i in range(len(data)):
        body_t = data[i]['transl']
        body_r = data[i]['global_orient']
        body_shape=data[i]['betas']
        body_pose = data[i]['pose_embedding']
        body_lhp = data[i]['left_hand_pose']
        body_rhp = data[i]['right_hand_pose']
        body = np.concatenate([body_t, body_r, body_shape, 
                               body_pose, body_lhp, body_rhp
                               ],
                               axis=-1)
       
        dataset.append(body)
    #train_input=[]
    #train_gt=[]
    #test_input=[]
    #test_gt=[]
    #np.random.seed(2020)
    #sample=np.random.choice(len(data),int(len(data)*0.8),replace=False)
    input_list=[]
    gt_list=[]
   
    for i in range(len(dataset)):
        
        
        
        try:
            exist1=0
            exist2=0
            input1_list=[]
            for j in range(i,i-40,-1):
                if 0.2<np.mean((dataset[i][0][:3]-dataset[j][0][:3])**2)<10:
                    
                    input1_list.append(dataset[j])
                    input1_=dataset[j]
                    exist1=1
            
            #print('input1:',len(input1))
            #print('input1_:',len(input1_))
            input2_list=[]
            for k in range(i,i+40,1):
                if 0.2<np.mean((dataset[i][0][:3]-dataset[k][0][:3])**2)<10:
                    input2_list.append(dataset[k])
                    input2_=dataset[k]
                    exist2=1
            
            #print('input2:',len(input2))
            #print('input2_:',len(input2_))
            if exist1==1 and exist2==1:
                input1=random.choice(input1_list)
                input2=random.choice(input2_list)
                
            
                input_=torch.tensor(np.concatenate([input1,input2]), dtype=torch.float32)

                middle=torch.tensor(dataset[i]                                             
                               ,dtype=torch.float32)

            #if i in sample:
            #    train_input.append(input)
            #    train_gt.append(gt)
            #else:
            #    test_input.append(input)
            #    test_gt.append(gt)
            #if torch.max(np.isnan(input_))==0 and torch.max(np.isnan(middle))==0:
            if torch.max(np.isnan(input_))==0 and torch.max(np.isnan(middle))==0:
                input_list.append(input_)
                gt_list.append(middle)
            
            else:
                print(1)
        except:
            pass
       
    #print(len(input_list))
    #return train_input,train_gt,test_input,test_gt
    #return input_list,gt_list
    
    return input_list,gt_list,dataset

In [5]:
import json
sdf_path = scene_sdf_path='./data/prox/sdf' 
        
sub_data_testing=[]
sub_data_training=[]
total_file_list=sorted(os.listdir('./data/prox/PROXD/'))
testing_scene= ['MPH16','MPH1Library', 'N0SittingBooth','N3OpenArea']
for each in total_file_list:
    scene_name=each[:-9]
    with open(os.path.join(scene_sdf_path, scene_name+'.json')) as f:
        sdf_data = json.load(f)
        grid_min = np.array(sdf_data['min'])
        grid_max = np.array(sdf_data['max'])
        grid_dim = sdf_data['dim']
    sdf = np.load(os.path.join(scene_sdf_path, scene_name + '_sdf.npy')).reshape(grid_dim, grid_dim, grid_dim)
    scene_name = each[:-9]
    filelist = os.listdir('./data/prox/PROXD/'+each+'/results')
    filelist = sorted(filelist)
    temp_data_file = filelist
    data = []
    for i in range(len(temp_data_file)):
        f = open('./data/prox/PROXD/'+each+'/results/'+temp_data_file[i]+'/000.pkl','rb')
        temp = pickle.load(f)
        data.append(temp)
    sample_data = []
    for i in range(0,len(data),15):
        sample_data.append(data[i])
    input_list,middle_list,ds = create_dataset(sample_data)
    

    
    scene_mesh = o3d.io.read_triangle_mesh('./data/Proxe/scenes_downsampled/'+scene_name+'.ply')
    
    
    cam_ext_path = './data/prox/cam2world/'+scene_name+'.json'
    f = open(cam_ext_path,'r')
    contents = f.read();
    cam_ext = json.loads(contents)
    cam_ = np.linalg.inv(cam_ext)
    
    
    scene_verts = torch.tensor(np.asarray(scene_mesh.transform(cam_).vertices),dtype=torch.float32)
    
    grid_min = torch.tensor(np.array(sdf_data['min']),dtype=torch.float32)
    grid_max = torch.tensor(np.array(sdf_data['max']),dtype=torch.float32)
    if scene_name in testing_scene:
        for i in range(len(middle_list)):
            sub_data_testing.append([middle_list[i],each,scene_name,sdf,scene_verts,np.array(cam_ext),grid_min,grid_max])
        print('test',each,i)
    else:
        for i in range(len(middle_list)):
            sub_data_training.append([middle_list[i],each,scene_name,sdf,scene_verts,np.array(cam_ext),grid_min,grid_max])
        print('train',each,i)

train BasementSittingBooth_00142_01 66
train BasementSittingBooth_00145_01 106


C:\Users\jiash\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in greater
C:\Users\jiash\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in greater


train BasementSittingBooth_03452_01 113
train MPH112_00034_01 83
train MPH112_00150_01 77
train MPH112_00151_01 46
train MPH112_00157_01 43
train MPH112_00169_01 78
train MPH112_03515_01 60
train MPH11_00034_01 105
train MPH11_00150_01 106
train MPH11_00151_01 58
train MPH11_03515_01 88
test MPH16_00157_01 79
test MPH16_03301_01 62
test MPH1Library_00034_01 108
train MPH8_00168_01 162
train MPH8_03301_01 96
test N0SittingBooth_00162_01 59
test N0SittingBooth_00169_01 34
test N0SittingBooth_00169_02 43
test N0SittingBooth_03301_01 33
test N0SittingBooth_03403_01 51
train N0Sofa_00034_01 163
train N0Sofa_00034_02 57
train N0Sofa_00141_01 112
train N0Sofa_00145_01 101
train N3Library_00157_01 25
train N3Library_00157_02 8
train N3Library_03301_01 16
train N3Library_03301_02 4
train N3Library_03375_01 34
train N3Library_03375_02 450
train N3Library_03403_01 5
train N3Library_03403_02 26
train N3Office_00034_01 98
train N3Office_00139_01 50
train N3Office_00139_02 109
train N3Office_00150_0

In [ ]:
np.save('./data/sub_data_training.npy',np.array(sub_data_training))

In [13]:
np.save('./data/sub_data_testing.npy',np.array(sub_data_training))